# Exemplo 02 - Mostrar Palavras

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.\
        builder.\
        appName("pyspark-notebook").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        getOrCreate()

In [ ]:
sc = spark.sparkContext

### Ler Arquivo do PDF

In [ ]:
# Execute esta célula apenas uma vez
!pip install textract

In [ ]:
import textract

In [ ]:
text = textract.process("arquivo.pdf", method='pdfminer')
texto_corr = text.decode('utf-8')

In [ ]:
file_in = sc.parallelize([texto_corr])

In [ ]:
# Adiciona o operador add
from operator import add

In [ ]:
# Contar quantos caracteres
chars = file_in.map(lambda s: len(s)).reduce(add)
print('Número de caracteres no arquivo: %s' % chars)

### Organizar a lista de palavras

In [ ]:
# importar o módulo regex
import re

In [ ]:
# Definir a lista de palavras indesejadas
palavras_indesejadas = ["404", "agosto", "julho", "aulapython", "minicursos", "count", "erad", "2021", "para", "como", "como", "possui", "uma", "que", "com", "são", "quando", "linha", "pode", "ser", "ERAD-CO"]

# Dividir cada linha em palavras e criar um RDD com todas as palavras
palavras = file_in.flatMap(lambda linha: linha.split(" "))

# Colocar todas as palavras em letras minúsculas
palavras = palavras.flatMap(lambda line: re.split('\W+', line.lower().strip()))

# Filtrar as palavras indesejadas
palavras = palavras.filter(lambda palavra: palavra not in palavras_indesejadas)

In [ ]:
# Filtrar apenas as palavras que não possuam caracteres
palavras = palavras.filter(lambda x: len(x) > 2)

# Fazer com que cada palavra tenha o valor 1
palavras = palavras.map(lambda w: (w,1))

# Reduzir para o total
palavras = palavras.reduceByKey(add)

In [ ]:
# Criar uma tupla (count, word) e ordenar de forma ascendente
tpalavras = palavras.map(lambda x:(x[1],x[0])).sortByKey(False)

# Pegar o Top 10
tpalavras.take(10)

In [ ]:
# Execute esta célula apenas uma vez
!pip3 install matplotlib

### Montar um gráfico

In [ ]:
# Importar a biblioteca necessária
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Obter os dados
dados = tpalavras.take(10)

# Inverter a ordem dos dados
dados = list(reversed(dados))

# Extrair os valores e rótulos da lista
values = [item[0] for item in dados]
labels = [item[1] for item in dados]

# Criar um gráfico de barras horizontais
plt.barh(range(len(dados)), values, align='center')
plt.yticks(range(len(dados)), labels)

# Definir os rótulos dos eixos
plt.xlabel('Contagem')
plt.ylabel('Palavra')

# Exibir o gráfico
plt.show()

In [ ]:
spark.sparkContext.stop()